In [1]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import rdkit
from rdkit import Chem
from rdkit.Chem import AllChem
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.linear_model import RidgeCV
from sklearn.linear_model import LassoCV
from sklearn.linear_model import ElasticNet
from sklearn.metrics import mean_squared_error

In [2]:
"""
Read in train and test as Pandas DataFrames
"""
# df_train = pd.read_csv("train.csv")
df_non_test = pd.read_csv("train.csv")
# df_train_full = df_non_test.sample(frac=0.8, random_state=200)
# df_validation = df_non_test.drop(df_train_full.index)

msk = np.random.rand(len(df_non_test)) < 0.8
df_train_full, df_validation = df_non_test[msk].copy(deep = True), df_non_test[~msk].copy(deep = True)

df_train = df_train_full.sample(n=6000)
df_test = pd.read_csv("test.csv")

In [3]:
print(len(df_train))
print(len(df_validation))
print(len(df_test))

6000
199609
824230


In [4]:
df_train.head()

,smiles,feat_001,feat_002,feat_003,feat_004,feat_005,feat_006,feat_007,feat_008,feat_009,...,feat_248,feat_249,feat_250,feat_251,feat_252,feat_253,feat_254,feat_255,feat_256,gap
731421,c1ccc2c(c1)sc1cc3cc(oc3cc21)-c1cccc2nsnc12,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.95
269616,c1sc(-c2cc3c4nsnc4c4ccc5cocc5c4c3c3ccccc23)c2c...,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.76
489822,[nH]1c2C=C[SiH2]c2c2sc3cc([se]c3c12)C1=CC=C[Si...,1.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.86
54518,[nH]1cccc1-c1ccc2cc3c(cc2c1)c1=C[SiH2]C=c1c1cc...,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.64
512168,C1=Cc2oc3c(sc4cc(-c5nccs5)c5cocc5c34)c2[SiH2]1,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.92


In [5]:
df_validation.head()

,smiles,feat_001,feat_002,feat_003,feat_004,feat_005,feat_006,feat_007,feat_008,feat_009,...,feat_248,feat_249,feat_250,feat_251,feat_252,feat_253,feat_254,feat_255,feat_256,gap
2,[nH]1c-2c([SiH2]c3cc(-c4scc5C=CCc45)c4nsnc4c-2...,1.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,...,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.49
6,c1ncc(s1)-c1cnc2c(c1)oc1c2ccc2ccccc12,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.91
18,c1cnc(s1)-c1ccc(cc1)-c1sc(c2Cccc12)-c1scc2occc12,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.08
29,c1cc2cc3c4c[nH]cc4c(cc3cc2o1)-c1cccc2=C[SiH2]C...,1.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.77
30,c1Cc(cc1)-c1sc(-c2sc(c3[SiH2]ccc23)-c2ccccc2)c...,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,...,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.84


In [6]:
df_test.head()

,Id,smiles,feat_001,feat_002,feat_003,feat_004,feat_005,feat_006,feat_007,feat_008,...,feat_247,feat_248,feat_249,feat_250,feat_251,feat_252,feat_253,feat_254,feat_255,feat_256
0,1,c1sc(-c2cnc3c(c2)c2nsnc2c2cc4cccnc4cc32)c2cc[n...,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2,[nH]1cccc1-c1cc2c3nsnc3c3c4sccc4[nH]c3c2s1,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,3,[nH]1c2cc(-c3ccc[se]3)c3nsnc3c2c2c3cscc3c3ccc4...,1.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,4,[nH]1c(cc2cnc3c(c12)c1=C[SiH2]C=c1c1ccc2=CCC=c...,1.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,5,c1sc(-c2sc(-c3sc(-c4scc5[se]ccc45)c4ccoc34)c3c...,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [7]:
#store gap values
Y_train = df_train.gap.values
Y_validation = df_validation.gap.values
#row where validation examples start
validation_idx = df_train.shape[0]
#row where testing examples start
test_idx = df_train.shape[0] + df_validation.shape[0]
#delete 'Id' column
df_test = df_test.drop(['Id'], axis=1)
#delete 'gap' column
df_train = df_train.drop(['gap'], axis=1)
df_validation = df_validation.drop(['gap'], axis=1)

In [ ]:
#extract 1024 features and add to molecules data
smiles_train_list = df_train['smiles']
smiles_validation_list = df_validation['smiles']
smiles_test_list = df_test['smiles']

def mol_to_objects(x): return Chem.MolFromSmiles('Cc1ccccc1')
m_train = list(map(mol_to_objects, smiles_train_list)) # list of molecule objects
m_validation = list(map(mol_to_objects, smiles_validation_list))
m_test = list(map(mol_to_objects, smiles_test_list))

def features_extract(x): return AllChem.GetMorganFingerprintAsBitVect(x,2,nBits=1024)
fp_train = list(map(features_extract, m_train)) #list of bit vectors
fp_validation = list(map(features_extract, m_validation))
fp_test = list(map(features_extract, m_test))

def bit_value(x): return [x.GetBit(i) for i in range(x.GetNumBits())]
fp_values_train = np.vstack(map(bit_value, fp_train)) # extract values
fp_values_validation = np.vstack(map(bit_value, fp_validation))
fp_values_test = np.vstack(map(bit_value, fp_test))

df_train = np.concatenate((df_train, fp_values_train),axis=1)
df_validation = np.concatenate((df_validation, fp_values_validation),axis=1)
df_test = np.concatenate((df_test, fp_values_test),axis=1)

In [ ]:
#DataFrame with all train and test examples so we can more easily apply feature engineering on
df_all = pd.concat((df_train, df_validation, df_test), axis=0)
df_all.head()

In [ ]:
"""
Example Feature Engineering

this calculates the length of each smile string and adds a feature column with those lengths
Note: this is NOT a good feature and will result in a lower score!
"""
#smiles_len = np.vstack(df_all.smiles.astype(str).apply(lambda x: len(x)))
#df_all['smiles_len'] = pd.DataFrame(smiles_len)

In [ ]:
#Drop the 'smiles' column
df_all = df_all.drop(['smiles'], axis=1)
vals = df_all.values
X_train = vals[:validation_idx]
X_validation = vals[validation_idx:test_idx]
X_test = vals[test_idx:]
print("Train features:", X_train.shape)
print("Train gap:", Y_train.shape)
print("Validation features:", X_validation.shape)
print("Validation gap:", Y_validation.shape)
print("Test features:", X_test.shape)

In [ ]:
#Linear Regression
LR = LinearRegression()
LR.fit(X_train, Y_train)
LR_pred = LR.predict(X_test)

# print 'LR Train R^2:', LR.score(X_train, Y_train)
# print 'LR Train RMSE:', np.sqrt(mean_squared_error(Y_train, LR.predict(X_train)))
print 'LR Validation R^2:', LR.score(X_validation, Y_validation)
print 'LR Validation RMSE:', np.sqrt(mean_squared_error(Y_validation, LR.predict(X_validation)))

In [ ]:
#Random Forest 
RF = RandomForestRegressor()
RF.fit(X_train, Y_train)
RF_pred = RF.predict(X_test)

# print('RF Train R^2:', RF.score(X_train, Y_train))
# print('RF Train RMSE:', np.sqrt(mean_squared_error(Y_train, RF.predict(X_train))))
print 'RF Validation R^2:', RF.score(X_validation, Y_validation) 
print 'RF Validation RMSE:', np.sqrt(mean_squared_error(Y_validation, RF.predict(X_validation)))

In [ ]:
#lambda values to test for Ridge and Lasso Regression
lambdas = [.001,.005,1,5,10,50,100,500,1000]
# lambdas = [.001,.005,.01,.05,.1,.5,1,5,10,50,100,250,500,750,1000]

In [ ]:
#Ridge Regression
Ridge = RidgeCV(alphas=lambdas, fit_intercept=True, cv=5)
Ridge.fit(X_train, Y_train)
Ridge_pred = Ridge.predict(X_test)

# print 'Ridge Train R^2:', Ridge.score(X_train, Y_train)
# print 'Ridge Train RMSE:', np.sqrt(mean_squared_error(Y_train, Ridge.predict(X_train))) 
print 'Ridge Validation R^2:', Ridge.score(X_validation, Y_validation) 
print 'Ridge Validation RMSE:', np.sqrt(mean_squared_error(Y_validation, Ridge.predict(X_validation)))

In [ ]:
#Lasso Regression
Lasso = LassoCV(alphas=lambdas, fit_intercept=True, cv=5)
Lasso.fit(X_train, Y_train)
Lasso_pred = Lasso.predict(X_test)

# print 'Lasso Train R^2:', Lasso.score(X_train, Y_train) 
# print 'Lasso Train RMSE:', np.sqrt(mean_squared_error(Y_train, Lasso.predict(X_train)))
print 'Lasso Validation R^2:', Lasso.score(X_validation, Y_validation) 
print 'Lasso Validation RMSE:', np.sqrt(mean_squared_error(Y_validation, Lasso.predict(X_validation)))

In [ ]:
#Elastic Net Regression
Elastic = ElasticNet(alpha=1.0, l1_ratio=0.5, fit_intercept=True)
Elastic.fit(X_train, Y_train)
Elastic_pred = Elastic.predict(X_test)

# print 'Elastic Net Train R^2:', Elastic.score(X_train, Y_train)
# print 'Elastic Net Train RMSE:', np.sqrt(mean_squared_error(Y_train, Elastic.predict(X_train)))
print 'Elastic Net Validation R^2:', Elastic.score(X_validation, Y_validation)
print 'Elastic Net Validation RMSE:', np.sqrt(mean_squared_error(Y_validation, Elastic.predict(X_validation)))

In [ ]:
def write_to_file(filename, predictions):
    with open(filename, "w") as f:
        f.write("Id,Prediction\n")
        for i,p in enumerate(predictions):
            f.write(str(i+1) + "," + str(p) + "\n")

In [ ]:
folder = os.path.dirname(os.getcwd()) + "/P1-regression/predictions/"

write_to_file(folder + "OLS.csv", LR_pred)
write_to_file(folder + "RF.csv", RF_pred)
write_to_file(folder + "Ridge.csv", Ridge_pred)
write_to_file(folder + "Lasso.csv", Lasso_pred)
write_to_file(folder + "ElasticNet.csv", Elastic_pred)